In [ ]:
import requests
import zipfile
import io
import re

# Set the URL of the shapefiles directory
url = "https://www2.census.gov/geo/tiger/TIGER2020/BG/"

# Send a GET request to the URL
response = requests.get(url)

# Extract the ZIP file URLs using regular expressions
zip_urls = re.findall(r'<a href="(.*\.zip)">', response.content.decode())

# Download and extract each ZIP file
for zip_url in zip_urls:
    # Send a GET request to the ZIP file URL
    response = requests.get(url + zip_url)
    
    # Extract the contents of the ZIP file
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
        # Loop through the files in the ZIP file
        found_shapefile = False
        for file_name in zip_file.namelist():
            # Check if the file is a shapefile
            if file_name.endswith(".shp") or file_name.endswith(".shx"):
                found_shapefile = True
                # Extract the file to a folder
                zip_file.extract(file_name, path="shapefiles")
        if not found_shapefile:
            print(f"No shapefiles found in {zip_url}")


In [1]:
import pandas as pd
import dask.dataframe as dd
import geopandas as gpd
from shapely.geometry import Point
import plotly.express as px

In [2]:
import geopandas as gpd
import os

# Set the path to the directory containing the shapefiles
shapefile_dir = "shapefiles"

# Create an empty list to hold the GeoDataFrames
gdf_list = []

# Loop through the shapefiles in the directory
for file_name in os.listdir(shapefile_dir):
    # Check if the file is a shapefile
    if file_name.endswith(".shp"):
        # Read the shapefile into a GeoDataFrame
        gdf = gpd.read_file(os.path.join(shapefile_dir, file_name))
        # Add the GeoDataFrame to the list
        gdf_list.append(gdf)

# Combine all the GeoDataFrames into a single GeoDataFrame
block_groups = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs=gdf.crs)

# Print the first few rows of the combined GeoDataFrame
print(block_groups.head())


                                            geometry
0  POLYGON ((-77.05014 38.90033, -77.05013 38.900...
1  POLYGON ((-77.03919 38.80050, -77.03913 38.800...
2  POLYGON ((-77.00540 38.86879, -77.00341 38.870...
3  POLYGON ((-76.98127 38.84662, -76.98098 38.846...
4  POLYGON ((-76.98334 38.85337, -76.98277 38.853...


In [3]:
chunksize = 100000

# Create an empty list to hold the dataframes
dfs = []

# Read the file in chunks
for df_chunk in pd.read_csv('q3_2020.csv', chunksize=chunksize):
    # Process the chunk as needed
    dfs.append(df_chunk)

# Concatenate the dataframes
census_data = pd.concat(dfs)
# Load the census block group data
#  = pd.read_csv("q3_2020.csv")

# Convert the latitude and longitude columns to a Point object
geometry = [Point(xy) for xy in zip(census_data['long'], census_data['lat '])]

# Create a GeoDataFrame with the census block group data and geometry column
gdf = gpd.GeoDataFrame(census_data, geometry=geometry, crs="EPSG:4326")

# Load the census block group shapefile
# block_groups = gpd.read_file("shapefile/tl_2020_us_state.shp")

In [4]:
import geopandas as gpd

# specify the path to the shapefile
shapefile_path = 'Datasets/tl_2016_us_county_tigerline_shapefiles/tl_2016_us_county.shp'

# read the shapefile using GeoPandas
region = gpd.read_file(shapefile_path)
region


,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,NaN,NaN,NaN,A,1477895811,10447360,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,NaN,NaN,NaN,A,680956787,61588406,+46.2946377,-123.4244583,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,NaN,NaN,NaN,A,6016761713,29147306,+34.3592729,-104.3686961,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,NaN,A,2169240199,22877180,+40.7835474,-096.6886584,"POLYGON ((-96.91060 40.95841, -96.91060 40.958..."
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,NaN,NaN,NaN,A,1489645187,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,13,123,00351260,13123,Gilmer,Gilmer County,06,H1,G4020,NaN,NaN,NaN,A,1103983377,12186837,+34.6904951,-084.4546507,"POLYGON ((-84.65478 34.66559, -84.65488 34.669..."
3229,27,135,00659513,27135,Roseau,Roseau County,06,H1,G4020,NaN,NaN,NaN,A,4329460632,16924718,+48.7610683,-095.8215042,"POLYGON ((-96.40466 48.80528, -96.40467 48.813..."
3230,28,089,00695768,28089,Madison,Madison County,06,H1,G4020,298,27140,NaN,A,1850058790,71143948,+32.6343703,-090.0341603,"POLYGON ((-90.09363 32.70763, -90.09360 32.707..."
3231,48,227,01383899,48227,Howard,Howard County,06,H1,G4020,NaN,13700,NaN,A,2333039139,8841781,+32.3034712,-101.4387720,"POLYGON ((-101.69227 32.27106, -101.69221 32.2..."


In [5]:
region_dict = {
    'Northeast': [42, 34, 36, 9, 25, 44, 50, 33, 23],
    'South': [48, 40, 5, 22, 28, 1, 21, 47, 13, 12, 45, 54, 51, 37, 11, 24, 10],
    'Midwest': [38, 46, 31, 20, 27, 19, 29, 55, 17, 18, 26, 39],
    'West': [53, 41, 6, 32, 16, 30, 4, 49, 56, 8, 35],
    'Alaska': [2],
    'Hawaii':[15]
}

def get_region(statefp):
    for region, states in region_dict.items():
        if int(statefp) in states:
            return region
    return None
for i in range(len(region)):
    region['STATEFP'][i] = get_region(region['STATEFP'][i])
region

/var/folders/5r/b69pxkqd4xscl2bkwmk9m5700000gn/T/ipykernel_51688/1957286325.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region['STATEFP'][i] = get_region(region['STATEFP'][i])


,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,Midwest,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,NaN,NaN,NaN,A,1477895811,10447360,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,West,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,NaN,NaN,NaN,A,680956787,61588406,+46.2946377,-123.4244583,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,West,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,NaN,NaN,NaN,A,6016761713,29147306,+34.3592729,-104.3686961,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,Midwest,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,NaN,A,2169240199,22877180,+40.7835474,-096.6886584,"POLYGON ((-96.91060 40.95841, -96.91060 40.958..."
4,Midwest,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,NaN,NaN,NaN,A,1489645187,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,South,123,00351260,13123,Gilmer,Gilmer County,06,H1,G4020,NaN,NaN,NaN,A,1103983377,12186837,+34.6904951,-084.4546507,"POLYGON ((-84.65478 34.66559, -84.65488 34.669..."
3229,Midwest,135,00659513,27135,Roseau,Roseau County,06,H1,G4020,NaN,NaN,NaN,A,4329460632,16924718,+48.7610683,-095.8215042,"POLYGON ((-96.40466 48.80528, -96.40467 48.813..."
3230,South,089,00695768,28089,Madison,Madison County,06,H1,G4020,298,27140,NaN,A,1850058790,71143948,+32.6343703,-090.0341603,"POLYGON ((-90.09363 32.70763, -90.09360 32.707..."
3231,South,227,01383899,48227,Howard,Howard County,06,H1,G4020,NaN,13700,NaN,A,2333039139,8841781,+32.3034712,-101.4387720,"POLYGON ((-101.69227 32.27106, -101.69221 32.2..."


In [6]:
region = region[['STATEFP','geometry']]
region

,STATEFP,geometry
0,Midwest,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,West,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,West,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,Midwest,"POLYGON ((-96.91060 40.95841, -96.91060 40.958..."
4,Midwest,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."
...,...,...
3228,South,"POLYGON ((-84.65478 34.66559, -84.65488 34.669..."
3229,Midwest,"POLYGON ((-96.40466 48.80528, -96.40467 48.813..."
3230,South,"POLYGON ((-90.09363 32.70763, -90.09360 32.707..."
3231,South,"POLYGON ((-101.69227 32.27106, -101.69221 32.2..."


In [7]:
gdf['geo'] = gdf['geometry']
gdf

,uid,S,origin_census_block_group,dis,lat,long,org_cen_blk,geometry,geo
0,-3048.82988,0,1.033021e+10,6990.000000,34.654954,-87.976741,10330210004,POINT (-87.97674 34.65495),POINT (-87.97674 34.65495)
1,-2713.70019,1,1.053971e+10,847.000000,31.016670,-87.491668,10539707003,POINT (-87.49167 31.01667),POINT (-87.49167 31.01667)
2,-2913.03082,2,1.055011e+10,4009.000000,33.963463,-85.769547,10550106022,POINT (-85.76955 33.96346),POINT (-85.76955 33.96346)
3,-2972.82399,3,1.000000e+11,5542.217391,39.269891,-75.702374,100010401001,POINT (-75.70237 39.26989),POINT (-75.70237 39.26989)
4,-2969.65018,4,1.000000e+11,5874.293478,39.229036,-75.700309,100010401002,POINT (-75.70031 39.22904),POINT (-75.70031 39.22904)
...,...,...,...,...,...,...,...,...,...
219391,-2994.41902,219910,9.015907e+10,3061.956522,41.663891,-71.870843,90159073002,POINT (-71.87084 41.66389),POINT (-71.87084 41.66389)
219392,-2998.40759,219911,9.015907e+10,3442.163043,41.678269,-71.941750,90159073003,POINT (-71.94175 41.67827),POINT (-71.94175 41.67827)
219393,-2996.12423,219912,9.015907e+10,2886.054348,41.652046,-71.932223,90159073004,POINT (-71.93222 41.65205),POINT (-71.93222 41.65205)
219394,-2997.05908,219913,9.015908e+10,4926.967391,41.730545,-71.819313,90159081001,POINT (-71.81931 41.73055),POINT (-71.81931 41.73055)


In [8]:
combined = gpd.sjoin(region, gdf, how="left", op='intersects')
combined

/Users/divyansh/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/var/folders/5r/b69pxkqd4xscl2bkwmk9m5700000gn/T/ipykernel_51688/2820720314.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4269
Right CRS: EPSG:4326

  combined = gpd.sjoin(region, gdf, how="left", op='intersects')


,STATEFP,geometry,index_right,uid,S,origin_census_block_group,dis,lat,long,org_cen_blk,geo
0,Midwest,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",79749.0,-4051.70689,79811.0,3.100000e+11,4234.500000,41.824927,-96.873018,3.103997e+11,POINT (-96.87302 41.82493)
0,Midwest,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",79748.0,-4072.22115,79810.0,3.100000e+11,1595.967391,41.988425,-96.984375,3.103997e+11,POINT (-96.98438 41.98842)
0,Midwest,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",79752.0,-4042.00434,79814.0,3.100000e+11,1339.304348,41.807669,-96.680930,3.103997e+11,POINT (-96.68093 41.80767)
0,Midwest,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",79751.0,-4046.24902,79813.0,3.100000e+11,914.739130,41.835285,-96.718573,3.103997e+11,POINT (-96.71857 41.83528)
0,Midwest,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",79753.0,-4046.20172,79815.0,3.100000e+11,811.739130,41.838959,-96.708949,3.103997e+11,POINT (-96.70895 41.83896)
...,...,...,...,...,...,...,...,...,...,...,...
3232,South,"POLYGON ((-82.59529 38.36978, -82.59515 38.369...",182584.0,-3167.29119,182751.0,5.410000e+11,2559.228261,38.382871,-82.518350,5.409902e+11,POINT (-82.51835 38.38287)
3232,South,"POLYGON ((-82.59529 38.36978, -82.59515 38.369...",182583.0,-3168.13445,182750.0,5.410000e+11,2569.184783,38.388500,-82.528218,5.409902e+11,POINT (-82.52822 38.38850)
3232,South,"POLYGON ((-82.59529 38.36978, -82.59515 38.369...",182582.0,-3168.62654,182749.0,5.410000e+11,2552.369565,38.396381,-82.524095,5.409901e+11,POINT (-82.52409 38.39638)
3232,South,"POLYGON ((-82.59529 38.36978, -82.59515 38.369...",182580.0,-3169.07313,182747.0,5.410000e+11,2073.369565,38.401744,-82.524199,5.409901e+11,POINT (-82.52420 38.40174)


In [9]:
combined = combined.dropna()
combined

,STATEFP,geometry,index_right,uid,S,origin_census_block_group,dis,lat,long,org_cen_blk,geo
0,Midwest,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",79749.0,-4051.70689,79811.0,3.100000e+11,4234.500000,41.824927,-96.873018,3.103997e+11,POINT (-96.87302 41.82493)
0,Midwest,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",79748.0,-4072.22115,79810.0,3.100000e+11,1595.967391,41.988425,-96.984375,3.103997e+11,POINT (-96.98438 41.98842)
0,Midwest,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",79752.0,-4042.00434,79814.0,3.100000e+11,1339.304348,41.807669,-96.680930,3.103997e+11,POINT (-96.68093 41.80767)
0,Midwest,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",79751.0,-4046.24902,79813.0,3.100000e+11,914.739130,41.835285,-96.718573,3.103997e+11,POINT (-96.71857 41.83528)
0,Midwest,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",79753.0,-4046.20172,79815.0,3.100000e+11,811.739130,41.838959,-96.708949,3.103997e+11,POINT (-96.70895 41.83896)
...,...,...,...,...,...,...,...,...,...,...,...
3232,South,"POLYGON ((-82.59529 38.36978, -82.59515 38.369...",182584.0,-3167.29119,182751.0,5.410000e+11,2559.228261,38.382871,-82.518350,5.409902e+11,POINT (-82.51835 38.38287)
3232,South,"POLYGON ((-82.59529 38.36978, -82.59515 38.369...",182583.0,-3168.13445,182750.0,5.410000e+11,2569.184783,38.388500,-82.528218,5.409902e+11,POINT (-82.52822 38.38850)
3232,South,"POLYGON ((-82.59529 38.36978, -82.59515 38.369...",182582.0,-3168.62654,182749.0,5.410000e+11,2552.369565,38.396381,-82.524095,5.409901e+11,POINT (-82.52409 38.39638)
3232,South,"POLYGON ((-82.59529 38.36978, -82.59515 38.369...",182580.0,-3169.07313,182747.0,5.410000e+11,2073.369565,38.401744,-82.524199,5.409901e+11,POINT (-82.52420 38.40174)


In [10]:
combined.drop(['geometry'], axis = 1, inplace = True) 
combined

/var/folders/5r/b69pxkqd4xscl2bkwmk9m5700000gn/T/ipykernel_51688/2010439475.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined.drop(['geometry'], axis = 1, inplace = True)


,STATEFP,index_right,uid,S,origin_census_block_group,dis,lat,long,org_cen_blk,geo
0,Midwest,79749.0,-4051.70689,79811.0,3.100000e+11,4234.500000,41.824927,-96.873018,3.103997e+11,POINT (-96.87302 41.82493)
0,Midwest,79748.0,-4072.22115,79810.0,3.100000e+11,1595.967391,41.988425,-96.984375,3.103997e+11,POINT (-96.98438 41.98842)
0,Midwest,79752.0,-4042.00434,79814.0,3.100000e+11,1339.304348,41.807669,-96.680930,3.103997e+11,POINT (-96.68093 41.80767)
0,Midwest,79751.0,-4046.24902,79813.0,3.100000e+11,914.739130,41.835285,-96.718573,3.103997e+11,POINT (-96.71857 41.83528)
0,Midwest,79753.0,-4046.20172,79815.0,3.100000e+11,811.739130,41.838959,-96.708949,3.103997e+11,POINT (-96.70895 41.83896)
...,...,...,...,...,...,...,...,...,...,...
3232,South,182584.0,-3167.29119,182751.0,5.410000e+11,2559.228261,38.382871,-82.518350,5.409902e+11,POINT (-82.51835 38.38287)
3232,South,182583.0,-3168.13445,182750.0,5.410000e+11,2569.184783,38.388500,-82.528218,5.409902e+11,POINT (-82.52822 38.38850)
3232,South,182582.0,-3168.62654,182749.0,5.410000e+11,2552.369565,38.396381,-82.524095,5.409901e+11,POINT (-82.52409 38.39638)
3232,South,182580.0,-3169.07313,182747.0,5.410000e+11,2073.369565,38.401744,-82.524199,5.409901e+11,POINT (-82.52420 38.40174)


In [11]:
combined.rename(columns={'geo': 'geometry'},
        inplace=True, errors='raise')
combined

/var/folders/5r/b69pxkqd4xscl2bkwmk9m5700000gn/T/ipykernel_51688/1024747189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined.rename(columns={'geo': 'geometry'},


,STATEFP,index_right,uid,S,origin_census_block_group,dis,lat,long,org_cen_blk,geometry
0,Midwest,79749.0,-4051.70689,79811.0,3.100000e+11,4234.500000,41.824927,-96.873018,3.103997e+11,POINT (-96.87302 41.82493)
0,Midwest,79748.0,-4072.22115,79810.0,3.100000e+11,1595.967391,41.988425,-96.984375,3.103997e+11,POINT (-96.98438 41.98842)
0,Midwest,79752.0,-4042.00434,79814.0,3.100000e+11,1339.304348,41.807669,-96.680930,3.103997e+11,POINT (-96.68093 41.80767)
0,Midwest,79751.0,-4046.24902,79813.0,3.100000e+11,914.739130,41.835285,-96.718573,3.103997e+11,POINT (-96.71857 41.83528)
0,Midwest,79753.0,-4046.20172,79815.0,3.100000e+11,811.739130,41.838959,-96.708949,3.103997e+11,POINT (-96.70895 41.83896)
...,...,...,...,...,...,...,...,...,...,...
3232,South,182584.0,-3167.29119,182751.0,5.410000e+11,2559.228261,38.382871,-82.518350,5.409902e+11,POINT (-82.51835 38.38287)
3232,South,182583.0,-3168.13445,182750.0,5.410000e+11,2569.184783,38.388500,-82.528218,5.409902e+11,POINT (-82.52822 38.38850)
3232,South,182582.0,-3168.62654,182749.0,5.410000e+11,2552.369565,38.396381,-82.524095,5.409901e+11,POINT (-82.52409 38.39638)
3232,South,182580.0,-3169.07313,182747.0,5.410000e+11,2073.369565,38.401744,-82.524199,5.409901e+11,POINT (-82.52420 38.40174)


In [12]:
combined.drop(['index_right'], axis = 1, inplace = True) 
combined

/var/folders/5r/b69pxkqd4xscl2bkwmk9m5700000gn/T/ipykernel_51688/2345446794.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined.drop(['index_right'], axis = 1, inplace = True)


,STATEFP,uid,S,origin_census_block_group,dis,lat,long,org_cen_blk,geometry
0,Midwest,-4051.70689,79811.0,3.100000e+11,4234.500000,41.824927,-96.873018,3.103997e+11,POINT (-96.87302 41.82493)
0,Midwest,-4072.22115,79810.0,3.100000e+11,1595.967391,41.988425,-96.984375,3.103997e+11,POINT (-96.98438 41.98842)
0,Midwest,-4042.00434,79814.0,3.100000e+11,1339.304348,41.807669,-96.680930,3.103997e+11,POINT (-96.68093 41.80767)
0,Midwest,-4046.24902,79813.0,3.100000e+11,914.739130,41.835285,-96.718573,3.103997e+11,POINT (-96.71857 41.83528)
0,Midwest,-4046.20172,79815.0,3.100000e+11,811.739130,41.838959,-96.708949,3.103997e+11,POINT (-96.70895 41.83896)
...,...,...,...,...,...,...,...,...,...
3232,South,-3167.29119,182751.0,5.410000e+11,2559.228261,38.382871,-82.518350,5.409902e+11,POINT (-82.51835 38.38287)
3232,South,-3168.13445,182750.0,5.410000e+11,2569.184783,38.388500,-82.528218,5.409902e+11,POINT (-82.52822 38.38850)
3232,South,-3168.62654,182749.0,5.410000e+11,2552.369565,38.396381,-82.524095,5.409901e+11,POINT (-82.52409 38.39638)
3232,South,-3169.07313,182747.0,5.410000e+11,2073.369565,38.401744,-82.524199,5.409901e+11,POINT (-82.52420 38.40174)


In [13]:
merged = gpd.sjoin(block_groups, combined, how="left", op='intersects')
merged

/Users/divyansh/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/var/folders/5r/b69pxkqd4xscl2bkwmk9m5700000gn/T/ipykernel_51688/3889239026.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  merged = gpd.sjoin(block_groups, combined, how="left", op='intersects')


,geometry,index_right,STATEFP,uid,S,origin_census_block_group,dis,lat,long,org_cen_blk
0,"POLYGON ((-77.05014 38.90033, -77.05013 38.900...",631.0,South,-2997.19037,3586.0,1.100000e+11,9490.489130,38.900567,-77.047473,1.100101e+11
1,"POLYGON ((-77.03919 38.80050, -77.03913 38.800...",631.0,South,-2989.54544,3590.0,1.100000e+11,1809.543478,38.813245,-77.023847,1.100101e+11
2,"POLYGON ((-77.00540 38.86879, -77.00341 38.870...",631.0,South,-2992.54692,3399.0,1.100000e+11,9856.086957,38.866810,-76.994920,1.100101e+11
3,"POLYGON ((-76.98127 38.84662, -76.98098 38.846...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"POLYGON ((-76.98334 38.85337, -76.98277 38.853...",631.0,South,-2990.72669,3402.0,1.100000e+11,1987.358696,38.851455,-76.978500,1.100101e+11
...,...,...,...,...,...,...,...,...,...,...
242743,"POLYGON ((-114.11791 46.36487, -114.11644 46.3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242744,"POLYGON ((-114.15529 46.22169, -114.15528 46.2...",2878.0,West,-5276.90478,79392.0,3.010000e+11,2586.032609,46.227760,-114.150128,3.008100e+11
242745,"POLYGON ((-114.16411 46.25464, -114.16387 46.2...",2878.0,West,-5279.63640,79389.0,3.010000e+11,1178.141304,46.247360,-114.160816,3.008100e+11
242745,"POLYGON ((-114.16411 46.25464, -114.16387 46.2...",2878.0,West,-5279.54778,79390.0,3.010000e+11,21132.728260,46.248263,-114.156671,3.008100e+11


In [14]:
merged = merged.dropna()
merged

,geometry,index_right,STATEFP,uid,S,origin_census_block_group,dis,lat,long,org_cen_blk
0,"POLYGON ((-77.05014 38.90033, -77.05013 38.900...",631.0,South,-2997.19037,3586.0,1.100000e+11,9490.489130,38.900567,-77.047473,1.100101e+11
1,"POLYGON ((-77.03919 38.80050, -77.03913 38.800...",631.0,South,-2989.54544,3590.0,1.100000e+11,1809.543478,38.813245,-77.023847,1.100101e+11
2,"POLYGON ((-77.00540 38.86879, -77.00341 38.870...",631.0,South,-2992.54692,3399.0,1.100000e+11,9856.086957,38.866810,-76.994920,1.100101e+11
4,"POLYGON ((-76.98334 38.85337, -76.98277 38.853...",631.0,South,-2990.72669,3402.0,1.100000e+11,1987.358696,38.851455,-76.978500,1.100101e+11
5,"POLYGON ((-76.98912 38.84472, -76.98888 38.844...",631.0,South,-2990.65403,3403.0,1.100000e+11,2203.793478,38.847759,-76.983952,1.100101e+11
...,...,...,...,...,...,...,...,...,...,...
242741,"POLYGON ((-114.08280 46.52802, -114.08272 46.5...",2878.0,West,-5298.64260,79378.0,3.010000e+11,4165.217391,46.475288,-114.009893,3.008100e+11
242742,"POLYGON ((-114.05628 46.57070, -114.05628 46.5...",2878.0,West,-5312.58531,79375.0,3.010000e+11,5716.913043,46.601703,-113.999809,3.008100e+11
242744,"POLYGON ((-114.15529 46.22169, -114.15528 46.2...",2878.0,West,-5276.90478,79392.0,3.010000e+11,2586.032609,46.227760,-114.150128,3.008100e+11
242745,"POLYGON ((-114.16411 46.25464, -114.16387 46.2...",2878.0,West,-5279.63640,79389.0,3.010000e+11,1178.141304,46.247360,-114.160816,3.008100e+11


In [15]:
merged.drop(['index_right'], axis = 1, inplace = True) 
merged

/var/folders/5r/b69pxkqd4xscl2bkwmk9m5700000gn/T/ipykernel_51688/2920029581.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.drop(['index_right'], axis = 1, inplace = True)


,geometry,STATEFP,uid,S,origin_census_block_group,dis,lat,long,org_cen_blk
0,"POLYGON ((-77.05014 38.90033, -77.05013 38.900...",South,-2997.19037,3586.0,1.100000e+11,9490.489130,38.900567,-77.047473,1.100101e+11
1,"POLYGON ((-77.03919 38.80050, -77.03913 38.800...",South,-2989.54544,3590.0,1.100000e+11,1809.543478,38.813245,-77.023847,1.100101e+11
2,"POLYGON ((-77.00540 38.86879, -77.00341 38.870...",South,-2992.54692,3399.0,1.100000e+11,9856.086957,38.866810,-76.994920,1.100101e+11
4,"POLYGON ((-76.98334 38.85337, -76.98277 38.853...",South,-2990.72669,3402.0,1.100000e+11,1987.358696,38.851455,-76.978500,1.100101e+11
5,"POLYGON ((-76.98912 38.84472, -76.98888 38.844...",South,-2990.65403,3403.0,1.100000e+11,2203.793478,38.847759,-76.983952,1.100101e+11
...,...,...,...,...,...,...,...,...,...
242741,"POLYGON ((-114.08280 46.52802, -114.08272 46.5...",West,-5298.64260,79378.0,3.010000e+11,4165.217391,46.475288,-114.009893,3.008100e+11
242742,"POLYGON ((-114.05628 46.57070, -114.05628 46.5...",West,-5312.58531,79375.0,3.010000e+11,5716.913043,46.601703,-113.999809,3.008100e+11
242744,"POLYGON ((-114.15529 46.22169, -114.15528 46.2...",West,-5276.90478,79392.0,3.010000e+11,2586.032609,46.227760,-114.150128,3.008100e+11
242745,"POLYGON ((-114.16411 46.25464, -114.16387 46.2...",West,-5279.63640,79389.0,3.010000e+11,1178.141304,46.247360,-114.160816,3.008100e+11


In [16]:
region_wise = merged.groupby('STATEFP')
region = region_wise.get_group('South')
region

,geometry,STATEFP,uid,S,origin_census_block_group,dis,lat,long,org_cen_blk
0,"POLYGON ((-77.05014 38.90033, -77.05013 38.900...",South,-2997.19037,3586.0,1.100000e+11,9490.489130,38.900567,-77.047473,1.100101e+11
1,"POLYGON ((-77.03919 38.80050, -77.03913 38.800...",South,-2989.54544,3590.0,1.100000e+11,1809.543478,38.813245,-77.023847,1.100101e+11
2,"POLYGON ((-77.00540 38.86879, -77.00341 38.870...",South,-2992.54692,3399.0,1.100000e+11,9856.086957,38.866810,-76.994920,1.100101e+11
4,"POLYGON ((-76.98334 38.85337, -76.98277 38.853...",South,-2990.72669,3402.0,1.100000e+11,1987.358696,38.851455,-76.978500,1.100101e+11
5,"POLYGON ((-76.98912 38.84472, -76.98888 38.844...",South,-2990.65403,3403.0,1.100000e+11,2203.793478,38.847759,-76.983952,1.100101e+11
...,...,...,...,...,...,...,...,...,...
236528,"POLYGON ((-90.99954 30.09679, -90.99885 30.096...",South,-2738.42835,46514.0,2.200000e+11,979.130435,30.094312,-90.994881,2.200503e+11
236529,"POLYGON ((-90.90410 30.25474, -90.90288 30.254...",South,-2749.37170,46476.0,2.200000e+11,1946.271739,30.249894,-90.888638,2.200503e+11
236531,"POLYGON ((-90.96564 30.32052, -90.96427 30.320...",South,-2757.23846,46482.0,2.200000e+11,3027.358696,30.313168,-90.958440,2.200503e+11
236533,"POLYGON ((-90.76734 29.61466, -90.76701 29.615...",South,-2687.69459,49644.0,2.210000e+11,1555.043478,29.611755,-90.764449,2.210900e+11


In [17]:
mapping = region

In [18]:
from pyproj import CRS
mapping.crs = 'EPSG:4326'  # Set the CRS to WGS84 if it's not already

# Estimate the UTM CRS and set the CRS to the estimated value
utm_crs = mapping.estimate_utm_crs()
mapping = mapping.to_crs(utm_crs)

In [19]:
# Simplify the geometry and set the CRS back to the original value
mapping['geometry'] = mapping.simplify(1000)
mapping = mapping.to_crs(CRS('EPSG:4326'))

In [27]:
import plotly.express as px
import plotly.io as pio
# Create a new column in df_agg based on distance ranges
bins = [0, 3855, 4815, 6047, float('inf')]
colors = ['red', 'yellow', 'lightgreen', 'darkgreen']
mapping['dis_range'] = pd.cut(mapping['dis'], bins=bins, labels=colors)
fig = px.choropleth_mapbox(
    mapping, 
    geojson=mapping.geometry, 
    locations=mapping.index,
    color='dis_range', 
    color_discrete_map={'red': 'red',
                        'blue': 'blue',
                        'lightgreen': 'lightgreen',
                        'darkgreen': 'darkgreen'},
    mapbox_style="carto-positron",  # Set the mapbox style to "carto-positron"
    zoom=1,
    opacity=0.5,
    labels={'dis':'Distance-Travelled'}
)

# Use plotly.io.show to generate the plot on a remote server and return the result to your local machine
# pio.show(fig, renderer='browser')


In [29]:
pio.show(fig, renderer='browser')

In [12]:
pio.write_html(fig, file='my_map.html', auto_open=True)

In [48]:
pio.write_image(fig, file='my_map.pdf')


In [25]:
church_df

,Worship Site,Address,Address 2,City,State,ZIP Code,Country,Latitude,Longitude,Diocese,Type,Language
0,Cathedral of the Immaculate Conception,125 Eagle Street,NaN,Albany,New York,12202-1797,United States,42.647453,-73.759478,Albany,Cathedral,English
1,St. Francis Chapel,145 Wolf Road,NaN,Colonie,New York,12205,United States,42.719231,-73.805428,Albany,Chapel,English
2,St. Therese,1 Wilton-Gansevoort Road,NaN,Gansevoort,New York,12831,United States,43.195728,-73.652176,Albany,Chapel,English
3,Christ Sun of Justice (Chapel +Cultural Center),2125 Burdett Ave,NaN,Troy,New York,12180,United States,42.731693,-73.672211,Albany,College/University,English
4,Siena College [Saint Mary of the Angels],515 Loudon Road,NaN,Loudonville,New York,12211,United States,42.717379,-73.753595,Albany,College/University,English
...,...,...,...,...,...,...,...,...,...,...,...,...
20337,St. Thomas the Apostle,4453 Warren Sharon Road,NaN,Vienna,Ohio,44473,United States,41.237794,-80.660466,Youngstown,Parish,English
20338,St. William,5411 Mahoning Ave N.W.,NaN,Warren,Ohio,44483,United States,41.302605,-80.848640,Youngstown,Parish,English
20339,Our Lady Comforter of the Afflicted,517 S. Belle Vista Ave,NaN,Youngstown,Ohio,44509,United States,41.094092,-80.688624,Youngstown,Shrine,English
20340,Holy Face Monastery,1697 NJ-3,NaN,Clifton,New Jersey,07013,United States,40.859207,-74.181857,NaN,Oratory-Other,English


In [23]:
import plotly.graph_objects as go
church_df = pd.read_csv('Worship Sites by Zip Code + Address.csv')

churches = go.Scattermapbox(
    lat=church_df['Latitude'],
    lon=church_df['Longitude'],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=5,
        color='blue'
    ),
    name='Churches'
)

# Add the scatter plot of the churches to the choropleth mapbox plot
# fig.add_trace(churches)

In [28]:
if not church_df.empty:
    fig.add_trace(go.Scattermapbox(
        lat=church_df['Latitude'],
        lon=church_df['Longitude'],
        mode='markers',
        marker=dict(
            size=4,
            color='blue'
        ),
        name='Church'
    ))

In [29]:
import plotly.offline as pyo
pyo.iplot(fig)